In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re 


MODEL_NAME_FOR_TEST = "camembert-base"
MAX_LEN_FOR_TEST = 160 
MODEL_SAVE_PATH_FOR_LOAD = "save_model_fr/camembert_coherence_final_model.bin"
TOKENIZER_SAVE_PATH_FOR_LOAD = "save_model_fr/camembert_coherence_tokenizer/"
DEVICE_FOR_TEST = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def clean_text_for_prediction(text):
    text = str(text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s\.\?,!àâéèêëîïôûùüçÀÂÉÈÊËÎÏÔÛÙÜÇ\']', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text


model_to_test = None
tokenizer_to_test = None
device_to_test = None
max_len_to_test = None
clean_text_func_to_use = None


tokenizer_to_test = AutoTokenizer.from_pretrained(TOKENIZER_SAVE_PATH_FOR_LOAD)


model_to_test = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME_FOR_TEST, num_labels=2)

# Charger les poids sauvegardés
model_to_test.load_state_dict(torch.load(MODEL_SAVE_PATH_FOR_LOAD, map_location=DEVICE_FOR_TEST))
model_to_test.to(DEVICE_FOR_TEST)
model_to_test.eval() 

max_len_to_test = MAX_LEN_FOR_TEST 
clean_text_func_to_use = clean_text_for_prediction
device_to_test = DEVICE_FOR_TEST 

print("Modèle et tokenizer chargés depuis le disque et prêts pour l'inférence.")


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modèle et tokenizer chargés depuis le disque et prêts pour l'inférence.


In [8]:
# --- Fonction de prédiction (utilise les variables _to_test) ---
def predict_coherence(question, answer, model, tokenizer, device, max_len, clean_fn):

    cleaned_question = clean_fn(question)
    cleaned_answer = clean_fn(answer)

    encoding = tokenizer.encode_plus(
        cleaned_question,
        cleaned_answer,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        prediction_idx = torch.argmax(probs, dim=1).cpu().item()
        probability_score = probs[0][prediction_idx].cpu().item()

    return prediction_idx, probability_score


In [21]:

# --- Exemples de test ---
test_samples = [
    {"question": "où puis-je trouver le lait ?", "answer": "dans le rayons des oeufs."},
    {"question": "tu manges du poulet ?", "answer": "le poulet est un menbre de ma famille"},
    {"question": "Je cherche du café moulu", "answer": "Le ciel est bleu et les oiseaux chantent."}, # Incohérent
    {"question": "Où sont les tomates en conserve ?", "answer": "Avez-vous pensé à regarder la météo ?"}, # Incohérent
    {"question": "Quel est le prix des pommes ?", "answer": "Elles sont à 2.50€ le kilo cette semaine."},
    {"question": "Avez-vous du pain complet ?", "answer": "Non, désolé, nous n'avons plus de pain complet aujourd'hui."},
    {"question": "Je voudrais des piles AA.", "answer": "Les piles se trouvent généralement près des caisses ou au rayon électronique."}
]

for i, sample in enumerate(test_samples):
    q = sample["question"]
    a = sample["answer"]

    print(f"\nExemple {i+1}:")
    print(f"  Question: \"{q}\"")
    print(f"  Réponse:  \"{a}\"")

    pred_label, pred_prob = predict_coherence(q, a,
                                                model_to_test,
                                                tokenizer_to_test,
                                                device_to_test,
                                                max_len_to_test,
                                                clean_text_func_to_use)


    coherence_status = "Cohérent" if pred_label == 1 else "Non Cohérent"
    print(f"  Prédiction: {coherence_status} (Label: {pred_label}, Confiance: {pred_prob:.4f})")



Exemple 1:
  Question: "où puis-je trouver le lait ?"
  Réponse:  "dans le rayons des oeufs."
  Prédiction: Non Cohérent (Label: 0, Confiance: 0.7411)

Exemple 2:
  Question: "tu manges du poulet ?"
  Réponse:  "le poulet est un menbre de ma famille"
  Prédiction: Non Cohérent (Label: 0, Confiance: 0.9247)

Exemple 3:
  Question: "Je cherche du café moulu"
  Réponse:  "Le ciel est bleu et les oiseaux chantent."
  Prédiction: Non Cohérent (Label: 0, Confiance: 0.9231)

Exemple 4:
  Question: "Où sont les tomates en conserve ?"
  Réponse:  "Avez-vous pensé à regarder la météo ?"
  Prédiction: Non Cohérent (Label: 0, Confiance: 0.9210)

Exemple 5:
  Question: "Quel est le prix des pommes ?"
  Réponse:  "Elles sont à 2.50€ le kilo cette semaine."
  Prédiction: Cohérent (Label: 1, Confiance: 0.9148)

Exemple 6:
  Question: "Avez-vous du pain complet ?"
  Réponse:  "Non, désolé, nous n'avons plus de pain complet aujourd'hui."
  Prédiction: Cohérent (Label: 1, Confiance: 0.9214)

Exemple 7:
